In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.feature_selection import SelectKBest, chi2, f_classif, mutual_info_classif
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier, VotingClassifier
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [2]:
dataset = pd.read_csv("Telco-Customer-Churn.csv")

In [3]:
dataset.drop("customerID", axis=1, inplace=True)

In [4]:
dataset['TotalCharges'] = pd.to_numeric(dataset['TotalCharges'], errors='coerce')
dataset['TotalCharges'].fillna(dataset['TotalCharges'].mean(), inplace=True)

In [5]:
dataset = dataset.apply(lambda x: x.map({'Yes': 1, 'No': 0}) if x.dtypes == 'object' and set(x.unique()) <= {'Yes', 'No'} else x)
dataset = pd.get_dummies(dataset, drop_first=True)

In [6]:
# Define features and target
X = dataset.drop("Churn", axis=1)
y = dataset["Churn"]

In [7]:
# Model training function
def train_models(X, y):
    models = {
        "Logistic": LogisticRegression(max_iter=1000),
        "SVM Linear": SVC(kernel="linear"),
        "SVM RBF": SVC(kernel="rbf"),
        "Naive Bayes": GaussianNB(),
        "KNN": KNeighborsClassifier(),
        "Decision Tree": DecisionTreeClassifier(),
        "Random Forest": RandomForestClassifier(),
        "Gradient Boosting": GradientBoostingClassifier(),
        "AdaBoost": AdaBoostClassifier(),
    }

    results = {}
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
    
    for name, model in models.items():
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        acc = accuracy_score(y_test, y_pred)
        results[name] = acc
    
    return results

In [8]:
# Chi2
X_chi2 = MinMaxScaler().fit_transform(X)
X_chi2_selected = SelectKBest(score_func=chi2, k=10).fit_transform(X_chi2, y)
chi2_results = train_models(X_chi2_selected, y)

In [9]:
# ANOVA
X_anova = StandardScaler().fit_transform(X)
X_anova_selected = SelectKBest(score_func=f_classif, k=10).fit_transform(X_anova, y)
anova_results = train_models(X_anova_selected, y)

In [10]:
# Mutual Information
X_mutual = StandardScaler().fit_transform(X)
X_mutual_selected = SelectKBest(score_func=mutual_info_classif, k=10).fit_transform(X_mutual, y)
mutual_results = train_models(X_mutual_selected, y)

In [11]:
# Combine and compare
final_results = pd.DataFrame({
    "Chi2": chi2_results,
    "ANOVA": anova_results,
    "Mutual_Info": mutual_results
})

print(final_results.T)  # Transposed view for better readability

             Logistic  SVM Linear   SVM RBF  Naive Bayes       KNN  \
Chi2         0.760931    0.727995  0.760931     0.592845  0.734242   
ANOVA        0.799546    0.802385  0.797842     0.592845  0.778535   
Mutual_Info  0.802385    0.796139  0.796706     0.616127  0.769449   

             Decision Tree  Random Forest  Gradient Boosting  AdaBoost  
Chi2              0.760931       0.760931           0.760931  0.760931  
ANOVA             0.792164       0.790460           0.802385  0.792164  
Mutual_Info       0.724588       0.777967           0.796706  0.792731  


In [12]:
#From the above looks live mutual information Logistic is the best feature selection

In [13]:
max_value = final_results.values.max()
max_location = final_results.stack().idxmax()
feature_method, model_name = max_location
print(f"✅ Best model is '{model_name}' with '{feature_method}' feature selection. Accuracy: {max_value:.4f}")

✅ Best model is 'Mutual_Info' with 'Logistic' feature selection. Accuracy: 0.8024
